In [30]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

import pandas as pd
import numpy as np

In [31]:
file_list = ["faces/non_celebrities/female/attractive/image_1.jpg",
            "faces/non_celebrities/female/attractive/image_2.jpg"]

In [32]:
def face_mesh_processor(image_path):
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        min_detection_confidence=0.5) as face_mesh:
        image = cv2.imread(image_path)
        # Convert the BGR image to RGB before processing.
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        return results

In [33]:
results_1 = face_mesh_processor("faces/non_celebrities/female/attractive/image_1.jpg")
results_2 = face_mesh_processor("faces/non_celebrities/female/attractive/image_2.jpg")

In [37]:
def landmark_processor(results):
    string = str(results.multi_face_landmarks[0])
    string = string.replace('\nlandmark ', '').replace('\n  ', '').replace('\n  ', '').replace('x: ', '').replace('y: ', ', ')
    string = string.replace('z: ', ', ').replace('\n', '').replace('landmark', '').replace(' ', '')
    string = string.split('{')[1:]
    string = [item.replace('}', '') for item in string]
    landmark_list = []
    for item in string:
        sub_list = item.split(',')
        sub_list = [float(x) for x in sub_list]
        landmark_list.append(sub_list)
    df = pd.DataFrame(landmark_list).rename(columns = {0:'x', 1:'y', 2:'z'})
    df['x'] = df['x'].astype(int)
    df['y'] = df['y'].astype(int)
    return df

In [38]:
df_1 = landmark_processor(results_1)
df_2 = landmark_processor(results_2)
df_1.shape, df_2.shape

((468, 3), (468, 3))

In [41]:
def landmark_projector(landmarks, image_path):
    index = 0
    for x_axis, y_axis in zip(landmarks['x'], landmarks['y']):
        image = cv2.imread(image_path)
        cv2.circle(image,(x_axis, y_axis), 3,(0, 255, 0), thickness=2, lineType=cv2.LINE_8 )
        cv2.imshow("landmark_" + str(index), image)
        cv2.waitKey(0)
        index += 1

In [ ]:
landmark_projector(df_2, "faces/non_celebrities/female/attractive/image_2.jpg")